In [2]:
from importlib import reload
import notebooks.amazon_review_analysis
reload(notebooks.amazon_review_analysis)
from notebooks.amazon_review_analysis import *
categories = [
    "All_Beauty",
    "Amazon_Fashion",
    "Appliances",
    "Arts_Crafts_and_Sewing",
    "Automotive",
    "Baby_Products",
    "Beauty_and_Personal_Care",
    "Books",
    "CDs_and_Vinyl",
    "Cell_Phones_and_Accessories",
    "Clothing_Shoes_and_Jewelry",
    "Digital_Music",
    "Electronics",
    "Gift_Cards",
    "Grocery_and_Gourmet_Food",
    "Handmade_Products",
    "Health_and_Household",
    "Health_and_Personal_Care",
    "Home_and_Kitchen",
    "Industrial_and_Scientific",
    "Kindle_Store",
    "Magazine_Subscriptions",
    "Movies_and_TV",
    "Musical_Instruments",
    "Office_Products",
    "Patio_Lawn_and_Garden",
    "Pet_Supplies",
    "Software",
    "Sports_and_Outdoors",
    "Subscription_Boxes",
    "Tools_and_Home_Improvement",
    "Toys_and_Games",
    "Video_Games"
]
    
    # Load metadata
category = "All_Beauty"
df = load_amazon_data_to_df(category)



2025-01-14 15:38:10.995 | INFO     | notebooks.amazon_review_analysis:load_amazon_data_to_df:27 - Loading All_Beauty dataset...
2025-01-14 15:38:23.564 | INFO     | notebooks.amazon_review_analysis:load_amazon_data_to_df:41 - Loaded 112590 records


In [3]:
df['price'] = pd.to_numeric(df['price'], errors='coerce')
df.head()

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,subtitle,author
0,All Beauty,"Howard LC0008 Leather Conditioner, 8-Ounce (4-...",4.8,10,[],[],NaN,"{'hi_res': [None, 'https://m.media-amazon.com/...","{'title': [], 'url': [], 'user_id': []}",Howard Products,[],"{""Package Dimensions"": ""7.1 x 5.5 x 3 inches; ...",B01CUPMQZE,None,None,None
1,All Beauty,Yes to Tomatoes Detoxifying Charcoal Cleanser ...,4.5,3,[],[],NaN,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Yes To,[],"{""Item Form"": ""Powder"", ""Skin Type"": ""Acne Pro...",B076WQZGPM,None,None,None
2,All Beauty,Eye Patch Black Adult with Tie Band (6 Per Pack),4.4,26,[],[],NaN,"{'hi_res': [None, None], 'large': ['https://m....","{'title': [], 'url': [], 'user_id': []}",Levine Health Products,[],"{""Manufacturer"": ""Levine Health Products""}",B000B658RI,None,None,None
3,All Beauty,"Tattoo Eyebrow Stickers, Waterproof Eyebrow, 4...",3.1,102,[],[],NaN,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Cherioll,[],"{""Brand"": ""Cherioll"", ""Item Form"": ""Powder"", ""...",B088FKY3VD,None,None,None
4,All Beauty,Precision Plunger Bars for Cartridge Grips – 9...,4.3,7,"[Material: 304 Stainless Steel; Brass tip, Len...",[The Precision Plunger Bars are designed to wo...,NaN,"{'hi_res': [None], 'large': ['https://m.media-...","{'title': [], 'url': [], 'user_id': []}",Precision,[],"{""UPC"": ""644287689178""}",B07NGFDN6G,None,None,None


In [4]:
def analyze(df):
    # Analyze category popularity
    results = analyze_category_popularity(df)

    print("\nCategory Metrics:")
    print(json.dumps(results['category_metrics'], indent=2))

    print("\nRating Distribution (%):")
    print(json.dumps(results['rating_distribution'], indent=2))

    print(f"\nMissing Prices: {results['missing_prices']}")
    print(f"Total Products: {results['total_products']}")

    # Analyze details distribution
    details_analysis = analyze_details_distribution(df)

    # Print results
    for field, stats in details_analysis.items():
        print(f"\n{field}:")
        print(f"Coverage: {stats['coverage_percentage']}")
        print(f"Unique values: {stats['unique_values']}")
        print("\nTop values:")
        for value, count in stats['top_values'].items():
            print(f"  {value}: {count}")

    # Analyze media presence
    media_stats = analyze_media_presence(df)
    print("\nMedia Presence:")
    print(f"Products with images: {media_stats['image_metrics']['products_with_images']['percentage']}%")
    print(f"Average images per product: {media_stats['image_metrics']['image_distribution']['mean']}")
    print(f"Products with videos: {media_stats['video_metrics']['products_with_videos']['percentage']}%")

    # Analyze brand distribution
    brand_dist = analyze_brand_distribution(df, min_products=10)

    # Print formatted analysis
    print(json.dumps(brand_dist, indent=2))

    # Analyze brand performance
    brand_perf = analyze_brand_performance(df, min_reviews=50)


    # Overall metrics
    metrics = brand_perf['overall_metrics']
    stats = metrics['review_statistics']
    dist = brand_perf['rating_distribution']

    # Analyze brand performance
    brand_perf = analyze_brand_performance(df, min_reviews=50)

    def format_price(price: Optional[float]) -> str:
        return f"${price:.2f}" if price else "N/A"

    # Create formatted output
    output = [
        "Brand Performance Analysis",
        "=" * 50,
        "\nOverall Metrics:",
        f"Total Brands: {brand_perf['overall_metrics']['total_brands']:,}",
        f"Brands Analyzed (≥50 reviews): {brand_perf['overall_metrics']['brands_analyzed']:,}",
        f"Mean Reviews per Brand: {brand_perf['overall_metrics']['review_statistics']['mean_reviews_per_brand']:,.1f}",
        f"Median Reviews per Brand: {brand_perf['overall_metrics']['review_statistics']['median_reviews_per_brand']:,.1f}",
        
        "\nRating Distribution:",
        f"Mean Rating: {brand_perf['rating_distribution']['mean']}",
        f"Median Rating: {brand_perf['rating_distribution']['median']}",
        f"Rating Std Dev: {brand_perf['rating_distribution']['std']}",
        
        "\nTop Brands by Review Count:"
    ]

    # Add top brands details
    for brand, metrics in list(brand_perf['top_brands'].items())[:10]:
        brand_details = [
            f"\n{brand}:",
            f"  Total Reviews: {metrics['total_reviews']:,}",
            f"  Average Rating: {metrics['average_rating']}",
            f"  Products: {metrics['product_count']}",
            f"  Average Price: {format_price(metrics['avg_price'])}",
            f"  Median Price: {format_price(metrics['median_price'])}"
        ]
        output.extend(brand_details)

    # Print the formatted output
    print('\n'.join(output))

    print("\nOverall Brand Metrics:")
    print(f"Total brands: {brand_perf['overall_metrics']['total_brands']}")
    print(f"Brands analyzed: {brand_perf['overall_metrics']['brands_analyzed']}")
    print(f"Mean reviews per brand: {brand_perf['overall_metrics']['review_statistics']['mean_reviews_per_brand']}")

    print("\nTop Brands by Review Count:")
    for brand, metrics in brand_perf['top_brands'].items():
        print(f"\n{brand}:")
        print(f"Total Reviews: {metrics['total_reviews']:,}")
        print(f"Average Rating: {metrics['average_rating']}")
        print(f"Products: {metrics['product_count']}")
        if metrics['avg_price']:
            print(f"Average Price: ${metrics['avg_price']}")

In [5]:
#category = "Cell_Phones_and_Accessories"
#df = load_amazon_data_to_df(category)
df = df.dropna(subset=['price'])
analyze(df)


Category Metrics:
{
  "average_rating_mean": {
    "All Beauty": 4.17,
    "Premium Beauty": 4.32
  },
  "rating_number_mean": {
    "All Beauty": 107.65,
    "Premium Beauty": 342.91
  },
  "price_mean": {
    "All Beauty": 27.09,
    "Premium Beauty": 48.36
  },
  "price_median": {
    "All Beauty": 16.95,
    "Premium Beauty": 30.0
  },
  "price_std": {
    "All Beauty": 50.36,
    "Premium Beauty": 59.49
  }
}

Rating Distribution (%):
{
  "4 stars": 54.63,
  "5 stars": 31.09,
  "3 stars": 11.07,
  "2 stars": 1.67,
  "1 star": 1.55
}

Missing Prices: 0
Total Products: 17704

Number of Items:
Coverage: 28.84%
Unique values: 47

Top values:
  1: 3590
  2: 652
  3: 300
  6: 136
  4: 131
  5: 57
  12: 52
  10: 45
  8: 21
  7: 15

Shape:
Coverage: 1.59%
Unique values: 19

Top values:
  Oblong: 144
  Rectangular: 37
  Round: 34
  Flat: 19
  Oval: 16
  Square: 9
  Barrel: 8
  Irregular: 4
  Bottle: 1
  Star: 1

Country of Origin:
Coverage: 3.56%
Unique values: 39

Top values:
  USA: 269
